In [1]:
import fitz  # PyMuPDF
import io
from PIL import Image
import os
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTImage, LTFigure,LTTextBox
import pandas as pd
import pdfplumber
import requests
import PyPDF2
from PyPDF2 import PageObject
import time

In [23]:
def extract_paragraph_from_pdf(pdf_path,start_word,end_word):
    pdf = fitz.open(pdf_path)
    paragraph =''
    for page_num in range(pdf.page_count):
        page = pdf.load_page(page_num)
        text = page.get_text("text")
        print(page_num,"=======================")
        # print(text)
        # 搜索 'Abstract' 关键字的位置
        start_idx = text.lower().find(start_word)
        print(start_idx)
        if start_idx != -1:
            
            # 如果找到了 'Abstract'，那么继续搜索 'Introduction' 作为结束位置
            end_idx = text.lower().find(end_word, start_idx)
            if end_idx == -1:
                # 如果没有找到 'Introduction'，则提取从 'Abstract' 开始的其余文本
                end_idx = len(text)
                
                paragraph = text[start_idx:end_idx].strip()
            else:
                paragraph = text[start_idx:].strip()
    if len(paragraph)==0 :
        print('Can not find introduction!')
    pdf.close()
    print(paragraph)
    return paragraph

In [16]:
def extract_images_and_text_below(pdf_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    pdf = fitz.open(pdf_path)
    pdf_img = pdfplumber.open(pdf_path) 
    output_imgs = []
    output_texts = []
    for page_num, page_layout in enumerate(extract_pages(pdf_path)):
        if (page_num ==0):
            continue
        page = pdf_img.pages[page_num]
        im = page.to_image(resolution=300)  # 高分辨率转换页面为图片
        pil_image = im.original
        pil_height = pil_image.height
        images = []
        text_blocks = []
        

        # 获取图像和文本块的位置信息
        for element in page_layout:
            # print(element)
            if isinstance(element, LTFigure):
                bbox = (element.bbox[0], element.bbox[1], element.bbox[2], element.bbox[3])
                bbox = element.bbox
                bbox_adjust = [0,0,0,0]
                for i in range(4):
                    bbox_adjust[i] = bbox[i] * pil_height /  page.height
                pil_bbox = (bbox_adjust[0], pil_height - bbox_adjust[3], bbox_adjust[2], pil_height - bbox_adjust[1])
                # 在 PIL.Image 对象上裁剪图片
                cropped_im = pil_image.crop(pil_bbox)
                images.append((element.bbox,cropped_im))
            elif isinstance(element, LTTextBox):
                text_blocks.append((element.bbox, element.get_text()))
        page = pdf[page_num]
        for i, (img_bbox,img) in enumerate(images):
            # 找到距离图片最近的文本块
            nearest_text = None
            min_distance = float('inf')
            img_x0, img_y0, img_x1, img_y1 = img_bbox
            for text_bbox, text_content in text_blocks:
                text_x0, text_y0, text_x1, text_y1 = text_bbox
                if text_y1 <= img_y0:  # 文本块位于图像下方
                    distance = (img_y0 - text_y1) + abs(img_x0-text_x0)   # 计算文本块和图像的垂直距离
                    if distance < min_distance:
                        min_distance = distance
                        nearest_text = text_content
            if nearest_text:
                # 使用 PDFMiner 提取并保存最近的文本块
                text_filename = f"page{page_num + 1}_text_below_img{i + 1}.txt"
                text_filepath = os.path.join(output_folder, text_filename)
                with open(text_filepath, "w", encoding="utf-8") as f:
                    f.write(nearest_text)

                img_filename = f"page{page_num + 1}_img{i + 1}.png"
                img_filepath = os.path.join(output_folder, img_filename)
                img.save(img_filepath)

                output_texts.append(nearest_text)
                output_imgs.append(img)
    pdf.close()
    print("==============================")
    print(f"{len(output_imgs)} figures and legends extracted.")
    return output_texts,output_imgs

output_folder = "extracted_images"
extract_images_and_text_below('new_pdfs\PMC8998189.pdf',output_folder)

4 figures and legends extracted.


(['Figure 1. Hypoxic stimuli have opposing effects on UC-MSC proliferation. UC-MSCs were cultured\nunder different O2 and CoCl2 levels—control (21% O2), and moderate (5% O2 and 10 µM CoCl2)\nand severe (<1% O2 and 250 µM CoCl2) hypoxia—and cultured with INK-128, for 24 and 48 h.\n(A) Representative images of plated UC-MSCs treated under control and experimental conditions,\ncaptured in a phase-contrast microscope (10× magniﬁcation). (B) Total number of live cells counted\nafter 24 and 48 h incubation period, under control and experimental conditions, from a minimum\nof 3 independent experiments. Results are represented as means with SEM. Statistical signiﬁcance\nconsidered when * p < 0.05%, ** p < 0.01 and *** p < 0.001. (C) Representative ﬂow cytometry dot plot\nand the respective 24 h (D) and 48 h (F) quantiﬁcation of live, early apoptotic, late apoptotic, and\nnecrotic populations through Annexin V/PI staining. (E) Enhanced scale of the early apoptosis bars\nof graph D.\n',
  'Figur

In [4]:
introduction

'����������\n�������\nCitation: Moniz, I.; Ramalho-Santos,\nJ.; Branco, A.F. Differential Oxygen\nExposure Modulates Mesenchymal\nStem Cell Metabolism and\nProliferation through mTOR\nSignaling. Int. J. Mol. Sci. 2022, 23,\n3749. https://doi.org/10.3390/\nijms23073749\nAcademic Editor: Aleksandra\nKlimczak\nReceived: 22 February 2022\nAccepted: 25 March 2022\nPublished: 29 March 2022\nPublisher’s Note: MDPI stays neutral\nwith regard to jurisdictional claims in\npublished maps and institutional afﬁl-\niations.\nCopyright:\n© 2022 by the authors.\nLicensee MDPI, Basel, Switzerland.\nThis article is an open access article\ndistributed\nunder\nthe\nterms\nand\nconditions of the Creative Commons\nAttribution (CC BY) license (https://\ncreativecommons.org/licenses/by/\n4.0/).\n \nInternational Journal of \nMolecular Sciences\nArticle\nDifferential Oxygen Exposure Modulates Mesenchymal Stem\nCell Metabolism and Proliferation through mTOR Signaling\nInês Moniz 1\n, João Ramalho-Santos 1,2,*\n